In [1]:
from ultralytics import YOLO
from ultralytics.utils.ops import xyxy2xywh
import torch
import pickle
import shutil
import os
import re
from data_classes.custom_dataset import CustomImageDataset
DATA_PATH = './data/VOC2012/'
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Available accelerator: {DEVICE}\n')
if DEVICE == 'cuda':
    !nvidia-smi

Available accelerator: cuda

Fri Jan 26 00:47:28 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.40.06              Driver Version: 551.23         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3080 Ti     On  |   00000000:01:00.0  On |                  N/A |
|  0%   46C    P8             29W /  370W |    1231MiB /  12288MiB |     25%      Default |
|                                         |                        |                  N/A |
+------------------

In [2]:
data_path_annotations = './data/VOC2012/Annotations/'
data_path_images = './data/VOC2012/JPEGImages/'
yolo_data_path = './datasets/yolo_data/'

In [4]:
with open('./model/label_encoder.pkl', 'rb') as encoder_file:
    label_encoder = pickle.load(encoder_file)

In [4]:
train_df = CustomImageDataset(DATA_PATH, label_encoder, split='train').df.copy()
valid_df = CustomImageDataset(DATA_PATH, label_encoder, split='valid').df.copy()
test_df = CustomImageDataset(DATA_PATH, label_encoder, split='test').df.copy()

for df in [train_df, valid_df, test_df]:
    df[['xcenter', 'ycenter', 'box_width', 'box_height']] = xyxy2xywh(df[['xmin', 'ymin', 'xmax', 'ymax']].values)
    df['xcenter'] = df['xcenter'] / df['width']
    df['ycenter'] = df['ycenter'] / df['height']
    df['box_width'] = df['box_width'] / df['width']
    df['box_height'] = df['box_height'] / df['height']

train_df.head()

,name,label,width,height,xmin,ymin,xmax,ymax,xcenter,ycenter,box_width,box_height
0,2008_007231.jpg,18,375,500,1.0,259.0,365.0,500.0,0.488000,0.759000,0.970667,0.482000
1,2010_002860.jpg,17,500,375,1.0,141.0,500.0,373.0,0.501000,0.685333,0.998000,0.618667
2,2008_003108.jpg,14,191,500,69.0,56.0,191.0,500.0,0.680628,0.556000,0.638743,0.888000
3,2007_006673.jpg,3,500,333,294.0,110.0,324.0,196.0,0.618000,0.459459,0.060000,0.258258
4,2012_001778.jpg,14,375,500,219.0,225.0,323.0,440.0,0.722667,0.665000,0.277333,0.430000


In [5]:
for sample, df in [('train', train_df), ('val', valid_df), ('test', test_df)]:
    sample_path_imgs = os.path.join(yolo_data_path, 'images', sample)
    sample_path_labels = os.path.join(yolo_data_path, 'labels', sample)
    if not os.path.exists(sample_path_imgs):
        os.makedirs(sample_path_imgs)
    if not os.path.exists(sample_path_labels):
        os.makedirs(sample_path_labels)
    unique_images = df['name'].unique()
    for img in unique_images:
        img_df = df[df['name'] == img]
        labels_string = ''
        for row in img_df.iterrows():
            row_series = row[1]
            row_label = f'{row_series.label} {row_series.xcenter} {row_series.ycenter} {row_series.box_width} {row_series.box_height}\n'
            labels_string += row_label
        shutil.copy(os.path.join(data_path_images, img), os.path.join(yolo_data_path, 'images', sample))
        txt_name = re.sub(r'\.+[a-z]*', '.txt', img)
        with open(os.path.join(yolo_data_path, 'labels', sample, txt_name), 'w') as txt_label:
            txt_label.write(labels_string.rstrip('\n'))

In [2]:
yolo = YOLO('yolov8m.pt')
results = yolo.train(model='yolov8m.pt', data='voc.yaml', epochs=15, imgsz=512, batch=8, pretrained='yolov8m.pt', visualize=True)

New https://pypi.org/project/ultralytics/8.1.6 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.5 🚀 Python-3.11.5 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12288MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=voc.yaml, epochs=15, time=None, patience=50, batch=8, imgsz=512, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=yolov8m.pt, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=True, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_fr

2024-01-25 23:38:39.482125: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-25 23:38:39.482201: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-25 23:38:39.519334: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Overriding model.yaml nc=80 with nc=20

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytic

train: Scanning /home/deep_learning/Object detection/datasets/yolo_data/labels/train.cache... 14070 images, 0 backgrounds, 0 corrupt: 100%|██████████| 14070/14070 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...

                   all       4772       6021      0.328      0.582      0.358       0.28

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      15/15      2.94G     0.7283     0.8949      1.097          8        512: 100%|██████████| 1759/1759 [02:29<00:00, 11.76it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 299/299 [00:20<00:00, 14.57it/s]


                   all       4772       6021      0.323      0.598      0.356      0.284

15 epochs completed in 0.704 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train/weights/best.pt, 52.0MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.1.5 🚀 Python-3.11.5 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12288MiB)
Model summary (fused): 218 layers, 25851340 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 299/299 [00:19<00:00, 15.31it/s]


                   all       4772       6021      0.323      0.597      0.356      0.283
             aeroplane       4772        155       0.44      0.742      0.582      0.494
               bicycle       4772        127      0.352      0.583      0.413      0.325
                  bird       4772        192      0.352      0.677      0.478      0.372
                  boat       4772        157       0.26      0.503      0.239      0.162
                bottle       4772        236      0.194      0.462      0.164      0.116
                   bus       4772         96      0.333      0.781      0.324      0.284
                   car       4772        370      0.276      0.487      0.295      0.233
                   cat       4772        192      0.609      0.839      0.706      0.607
                 chair       4772        501      0.185      0.395      0.161      0.106
                   cow       4772        122       0.29      0.648      0.336      0.259
           diningtabl

In [2]:
best_model_path = './runs/detect/train/weights/best.pt'
best_yolo = YOLO(best_model_path)